In [25]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import cv2

In [26]:
# Define paths to your dataset
image_folder = 'DRIVE/training/images'
label_folder = 'DRIVE/training/1st_manual'

# Get list of all files in the image and label directories
image_files = sorted(os.listdir(image_folder))
label_files = sorted(os.listdir(label_folder))



In [32]:
# Function to preprocess images and labels
def preprocess(image_path, label_path):
    # Convert the TensorFlow Tensor object to a string
    image_path = image_path.decode('utf-8')
    label_path = label_path.decode('utf-8')
    # Load image and label
    image = cv2.imread(image_path)
    label = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
    # Resize images and labels
    image_resized = cv2.resize(image, (512, 512))
    label_resized = cv2.resize(label, (512, 512))
    
    # Normalize images and labels
    image_normalized = image_resized.astype(np.float32) / 255.0
    label_normalized = label_resized.astype(np.float32) / 255.0
    
    return image_normalized, label_normalized



In [36]:
# Function to load the dataset into tf.data.Dataset
def load_dataset(image_folder, label_folder):
    # Create file paths
    image_paths = [os.path.join(image_folder, f) for f in image_files]
    label_paths = [os.path.join(label_folder, f) for f in label_files]
    # Create a TensorFlow dataset from the file paths
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, label_paths))
    
    # Map the preprocessing function to each element in the dataset
    dataset = dataset.map(lambda img_path, lbl_path: tf.numpy_function(
                          func=preprocess, inp=[img_path, lbl_path], 
                          Tout=[tf.float32, tf.float32]))
    
    # Shuffle and batch the dataset
    dataset = dataset.shuffle(buffer_size=len(image_paths))
    dataset = dataset.batch(4)  # Set batch size to 32, modify as needed
    
    return dataset



In [37]:
# Load the dataset
dataset = load_dataset(image_folder, label_folder)

In [38]:
# Example: iterate over the dataset
for batch in dataset:
    images, labels = batch
    print(f'Batch of images shape: {images.shape}')
    print(f'Batch of labels shape: {labels.shape}')

Batch of images shape: (4, 512, 512, 3)
Batch of labels shape: (4, 512, 512)
Batch of images shape: (4, 512, 512, 3)
Batch of labels shape: (4, 512, 512)
Batch of images shape: (4, 512, 512, 3)
Batch of labels shape: (4, 512, 512)
Batch of images shape: (4, 512, 512, 3)
Batch of labels shape: (4, 512, 512)
Batch of images shape: (4, 512, 512, 3)
Batch of labels shape: (4, 512, 512)
